In [1]:
from langchain_community.document_loaders import WebBaseLoader
import bs4

response = WebBaseLoader(
    web_path="https://en.wikipedia.org/wiki/Samantha_Ruth_Prabhu",
    bs_kwargs=dict(parse_only=bs4.SoupStrainer("p"))
)

extracted_text = response.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
split = splitter.split_documents(extracted_text)

In [3]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
database = Chroma.from_documents(split, embeddings)

C:\Users\abhay\AppData\Local\Temp\ipykernel_34680\2262757450.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\abhay\OneDrive\Desktop\python_LLM\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
query = "awards recived by samantha"
res = database.similarity_search(query)
res

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Samantha_Ruth_Prabhu'}, page_content='Samantha is the recipient of several awards, including four Filmfare Awards South, one Filmfare OTT Award, two Andhra Pradesh State Nandi Awards, four South Indian International Movie Awards, and three CineMAA Awards.'),
 Document(metadata={'source': 'https://en.wikipedia.org/wiki/Samantha_Ruth_Prabhu'}, page_content='"impressive", while DNA.com stated Samantha "has given a very subdued and mature performance sans drama".[74][75] Likewise, a reviewer from Silverscreen, suggested that she "delivers a performance that stands head and shoulders above her work in previous films".[76]'),
 Document(metadata={'source': 'https://en.wikipedia.org/wiki/Samantha_Ruth_Prabhu'}, page_content='kid to an adult is a treat to watch," while Rediff.com noted she is "pretty, graceful, and does an excellent job of conveying the emotions of a girl in love, especially during the school sequence, while concludin

In [5]:
from langchain_ollama import OllamaLLM

llm = OllamaLLM(model="deepseek-r1:1.5b")

In [6]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """
    you are an AI assist who will,
1. only answer from the available context, 
2. do not answer anything that is not there in the given context, 
3. if asked out of context, simply reply "out of context",

<context>
{context}
<context>

Question: {input}
    """
)

In [7]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm, prompt)

In [8]:
retriever = database.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001AB70E90A50>, search_kwargs={})

In [9]:
from langchain.chains import create_retrieval_chain

retrival_chain = create_retrieval_chain(retriever, document_chain)

In [11]:
response = retrival_chain.invoke({"input":"awards recived by samantha"})
print(response["answer"])

<think>
Okay, so I need to figure out how to answer this question about the awards Samantha received. Let me start by reading through the context provided.

The context mentions that Samantha has several awards, including four Filmfare Awards South, one Film fare OTT Award, two Andhra Pradesh State Nandi Awards, four South Indian International Movie Awards, and three CineMAA Awards. She also gave specific ratings: an "impressive" rating for one of her performances on DNA.com, while Rediff.com gave a higher rating saying she delivered a performance that stands above previous films. In another review by The Hindu, Samantha was praised for going further from her famous role and conveying character complexity well.

She has appeared in both Tamil and Telugu versions, receiving positive reviews and performing well commercially. She also received the Best Actress award at the 60th Filmfare Awards South after winning it for her performances in Neethane En Ponvasantham and Eega, making her the